In [9]:
import numpy as np 
import pandas as pd
import math
from PIL import Image
from tqdm import tqdm
import torchvision
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR, CosineAnnealingWarmRestarts
import copy
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision import transforms
from monai.networks.nets import ResNetFeatures, ViT, resnet18
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [19]:
#Creating a Transformation Object
transforms = v2.Compose([
    #Converting images to the size that the model expects
    v2.Resize(size=(224,224)),
    v2.ToTensor(), #Converting to tensor
    v2.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]) #Normalizing the data to the data that the ResNet18 was trained on
    
])

# Create custom dataset class
class XrayDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df.reset_index(drop=True)
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        xray_path = self.df.loc[idx, 'image_path']

        img = Image.open(xray_path).convert("RGB")
        # img = Image.open(xray_path)
        
        label = int(self.df.loc[idx, 'oa_label'])
    
        if self.transforms:
            img = self.transforms(img)

        return img, label

In [21]:
xray_path = '/home/feg48/kl_grading_project/xray/data/cropped_images/9689922_l.jpg'
img = Image.open(xray_path).convert("RGB")
img2 = transforms(img)

In [7]:
# model = MonaiResNet183DWrapper()
model = torchvision.models.resnet18(pretrained=True)
pretrained_weights = torch.load("../resnet_18_23dataset.pth")
model.load_state_dict({k: v for k, v in pretrained_weights.items() if "fc" not in k}, strict=False)
model.fc = torch.nn.Linear(in_features=512, out_features=1)

model = nn.DataParallel(model)
model = model.cuda()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-04)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)

/tmp/ipykernel_63346/3135094797.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_weights = torch.load("../resnet_18_23dataset.pth")


In [8]:
model

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [58]:
cv_best_models = []
cv_best_acc = []
batch_size = 32
epochs = 5

for fold in range(1, 2):
    xray_train_df = pd.read_csv(f"data/xray_train_cv{fold}.csv")
    xray_valid_df = pd.read_csv(f"data/xray_val_cv{fold}.csv")
    xray_train_dataset = XrayDataset(xray_train_df,  transforms=transforms)
    xray_train_dataloader = DataLoader(xray_train_dataset, batch_size=batch_size, shuffle=True)
    xray_val_dataset = XrayDataset(xray_valid_df, transforms=transforms)
    xray_val_dataloader = DataLoader(xray_val_dataset, batch_size=batch_size)
    
    model = torchvision.models.resnet18(pretrained=True)
    pretrained_weights = torch.load("../resnet_18_23dataset.pth")
    model.load_state_dict({k: v for k, v in pretrained_weights.items() if "fc" not in k}, strict=False)
    model.fc = torch.nn.Linear(in_features=512, out_features=1)
    model = nn.DataParallel(model)
    model = model.cuda()
    
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-04)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)
    
    best_val_acc = 0.
    best_model = None

    for epoch in range(epochs):
        print(f"Epoch {epoch}")
        losses = []
        correct = 0

        for img, label in tqdm(xray_train_dataloader, leave=False):
            optimizer.zero_grad()
            
            out = model(img.cuda()).squeeze(1)
            loss = loss_fn(out, label.float().cuda())
            losses.append(loss.item())
            loss.backward()
    
            correct += torch.sum((label.cuda() == (out > 0.5))).item()
            
            optimizer.step()
        print(f"Training Loss: {np.mean(losses)}")
        print(f"Training Accuracy: {correct/(len(xray_train_df))}")
    
        correct = 0
        # losses = 0
        for img, label in tqdm(xray_val_dataloader, leave=False):
            out = model(img.cuda()).squeeze(1)
    
            preds = (out > 0.5).float()
            loss = loss_fn(preds, label.float().cuda())
    
            # losses += loss.item() * img.size(0)
            correct += torch.sum(preds == label.cuda())
    
    
        final_acc = correct/len(xray_valid_df)
        print(f"Validation Accuracy: {final_acc}")
    
        if final_acc > best_val_acc:
            best_model = copy.deepcopy(model)
            best_val_acc = final_acc
    
        scheduler.step()
    
    losses = 0
    correct = 0
    for img, label in tqdm(xray_val_dataloader, leave=False):
        out = best_model(img.cuda()).squeeze(1)
    
        preds = (out > 0.5).float()
        loss = loss_fn(preds, label.float().cuda())
    
        losses += loss.item() * img.size(0)
        correct += torch.sum(preds == label.cuda())
    
    
    final_acc = correct/len(xray_valid_df)
    print(f"Fold {fold} Best Model Final Validation Accuracy: {final_acc}")

    cv_best_models.append(copy.deepcopy(best_model))
    cv_best_acc.append(final_acc)    

/home/feg48/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/feg48/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_63346/2818643565.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-mode

Epoch 0


Training Loss: 0.6537550299860245
Training Accuracy: 0.6179245283018868


Validation Accuracy: 0.6408106088638306
Epoch 1


Training Loss: 0.5855380363304522
Training Accuracy: 0.6811670160726765


...

Training Loss: 0.3493220183436431
Training Accuracy: 0.8385744234800838


Validation Accuracy: 0.6652690172195435
Epoch 4


Training Loss: 0.21602941529211386
Training Accuracy: 0.9135220125786163


Validation Accuracy: 0.6638714075088501


Fold 1 Best Model Final Validation Accuracy: 0.6652690172195435


In [59]:
max_acc = np.argmax([item.item() for item in cv_best_acc])

In [60]:
cv_best_acc

[tensor(0.6653, device='cuda:0')]

In [61]:
best_model = cv_best_models[max_acc]

In [62]:
xray_test_df= pd.read_csv("data/xray_test.csv")
xray_test_dataset = XrayDataset(xray_test_df, transforms=transforms)
xray_test_dataloader = DataLoader(xray_test_dataset, batch_size=batch_size)

labels = []
preds = []
for img, label in tqdm(xray_test_dataloader):
    out = best_model(img.cuda()).squeeze(1)
    preds.extend((out > 0.5).int().cpu().numpy())
    labels.extend(label.cpu().numpy())

    # correct += torch.sum(preds == label.cuda())

100%|██████████| 56/56 [00:04<00:00, 12.15it/s]


In [63]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Calculate metrics
accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds)
recall = recall_score(labels, preds)
f1 = f1_score(labels, preds)

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.68
Precision: 0.69
Recall: 0.47
F1 Score: 0.56


In [42]:
correct/len(xray_test_df)

tensor(0.6898, device='cuda:0')